In [1]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

C:\Users\dendluri\AppData\Roaming\Python\Python37\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Using TensorFlow backend.


In [2]:
X = pd.read_csv('../../data/problem-dataset/train.csv')

In [3]:
Y = X[['label']]

In [4]:
X = X.drop(["label"], axis=1)

In [5]:
X_test = pd.read_csv('../../data/problem-dataset/test.csv')

X_test = X_test.values.reshape(X_test.shape[0], 28, 28, 1)

In [6]:
X_train = X.values.reshape(X.shape[0], 28, 28, 1)

# Convert class vectors to binary class matrices
Y_train = tf.keras.utils.to_categorical(Y.values, 10)

In [7]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=42)

In [8]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

X_train /= 255
X_test /= 255
X_val /= 255

In [9]:
datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.2, 
        height_shift_range=0.2)

datagen.fit(X_train)

In [10]:
#Build model
model = tf.keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3, 3),
          activation='relu',
          input_shape=(28, 28, 1)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(layers.Dropout(0.2))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_v1 (Batc (None, 26, 26, 32)        128       
_________________________________________________________________
dropout (Dropout)            (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 24, 24, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPoo

In [11]:
model.compile(loss="categorical_crossentropy",
              optimizer="adadelta",
              metrics=['accuracy'])

In [12]:
model.fit(datagen.flow(X_train, Y_train, batch_size=128),
                    epochs=10,
                    validation_data=(X_val, Y_val),
                    verbose=1,
                    steps_per_epoch=X_train.shape[0] // 128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
296/296 [==============================] - 1045s 4s/step - loss: 0.7293 - acc: 0.7716 - val_loss: 5.1844 - val_acc: 0.1076
Epoch 2/10
296/296 [==============================] - 1231s 4s/step - loss: 0.2314 - acc: 0.9284 - val_loss: 0.6872 - val_acc: 0.7774
Epoch 3/10
296/296 [==============================] - 1199s 4s/step - loss: 0.1567 - acc: 0.9517 - val_loss: 0.0574 - val_acc: 0.9864
Epoch 4/10
296/296 [==============================] - 1064s 4s/step - loss: 0.1290 - acc: 0.9612 - val_loss: 0.0446 - val_acc: 0.9852
Epoch 5/10
296/296 [==============================] - 1110s 4s/step - loss: 0.1054 - acc: 0.9680 - val_loss: 0.0349 - val_acc: 0.9890
Epoch 6/10
296/296 [==============================] - 1069s 4s/step - loss: 0.0964 - acc: 0.9711 - val_loss: 0.1217 - val_acc: 0.9631
Epoch 7/10
296/296 [==============================] - 1036s 4s/step - loss: 0.0883 - acc: 0.9734 - val_loss: 0.0301 - val_acc: 0.9905
Epoch 8/10
296

In [13]:
Y_test = model.predict(X_test, batch_size=128, verbose=1)

28000/28000 [==============================] - 69s 2ms/sample


In [14]:
Y_test = np.argmax(Y_test, axis=1)

In [15]:
frame = pd.DataFrame({'Label': Y_test})
frame.insert(0, 'ImageId', range(1, 1 + len(frame)))
frame.to_csv('output.csv', index=False)